# HyLoRADA: Hybrid Low-Rank Adaptation with Landmark Memory

**Components**: rsLoRA + DoRA + LandmarkLoRA + PositionBias

| Component | Description | Params |
|-----------|-------------|--------|
| rsLoRA | α/√r scaling | 0 |
| DoRA | Magnitude normalization | ~86K |
| LandmarkLoRA | Context summaries (Novel) | ~14K |
| PositionBias | Position awareness | 64 |

In [ ]:
# Setup
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Clone repo (Kaggle)
import os
if os.path.exists('hylorada'):
    %cd hylorada
    !git pull
else:
    !git clone https://github.com/SadiaTabassum1216/hylorada.git
    %cd hylorada

In [ ]:
# Install dependencies
!pip install -q transformers datasets accelerate tqdm

In [ ]:
# Check GPU
import torch
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 1. Quick Demo - HyLoRADA Components

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from hylorada import HyLoRADAConfig, HyLoRADAModel, LandmarkLoRA

# Test LandmarkLoRA (Novel component)
lm = LandmarkLoRA(hidden_size=768, num_landmarks=8)
x = torch.randn(1, 64, 768)
y = lm(x)
print(f"LandmarkLoRA: {x.shape} -> {y.shape}")
print(f"LandmarkLoRA params: {sum(p.numel() for p in lm.parameters()):,}")

In [ ]:
# Load base model (GPT-2 for S²-Attn compatibility)
model_name = "openai-community/gpt2"
base_model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# HyLoRADA Config (Full Features)
config = HyLoRADAConfig(
    lora_rank=8,
    lora_alpha=16.0,
    use_dora_magnitude=True,    # DoRA
    landmark_enabled=True,      # LandmarkLoRA (Novel)
    num_landmarks=8,
    position_bias_enabled=True,
    # Long Context Features
    s2_attn_enabled=True,
    s2_group_size=2048,
)

print("Components:", config.get_component_status())

In [ ]:
# Apply HyLoRADA
model = HyLoRADAModel(base_model, config)
model.print_trainable_params()

## 2. Benchmark Section

In [ ]:
# 1. Quick Benchmark (GPT-2, 1024 context)
# Tests basic integration of LoRA vs HyLoRADA
!python run_benchmark.py --model openai-community/gpt2 --methods lora hylorada --epochs 1 --num_train 200 --max_length 1024

In [ ]:
# 2. HyLoRADA Long Context Run (GPT-2, 4096 context)
# This is the main experiment for your thesis/project

!python run_benchmark.py \
    --dataset longbench \
    --model openai-community/gpt2 \
    --max_length 4096 \
    --methods hylorada \
    --s2_attn \
    --train_embeddings \
    --train_norms \
    --rope_scaling_type linear \
    --rope_scaling_factor 4.0 \
    --epochs 1

In [ ]:
# 3. Full Comparison (All Baselines)
# Runs Baseline, LoRA, LoRaDA, LongLoRA, Sparse, and HyLoRADA
# WARNING: This may take longer to run

!python run_benchmark.py \
    --dataset longbench \
    --model openai-community/gpt2 \
    --max_length 4096 \
    --methods baseline lora lorada longlora sparse hylorada \
    --s2_attn \
    --train_embeddings \
    --train_norms \
    --rope_scaling_type linear \
    --rope_scaling_factor 4.0 \
    --epochs 1